# This is my first expirementational code with a GPT model
 I am following a tutorial made by Andrej Karpathy and plan on adding my own stuff throughout the process to improve my gained knowledge and understanding.


### First we start off by downloading this tinyshakespeare text.
 This wil be our starting data set for the transformer.

In [4]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-14 16:22:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2025-03-14 16:22:49 (7.47 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [1]:
with open('input.txt', 'r') as file:
    text = file.read()

In [2]:
print(len(text))

1115394


Here we are defining out vocabulary set. We do this by getting rid of the duplicates, then sorting it.

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Now we are doing the encoding and decoding. This is done in the following way:
we create lookup tables (dictionaries) from number to characters and vice-versa.
Then when encoding, we find the index of the characters in the given string, and return the array of indices.
As expected, for decoding, we get the collection of characters and join them together according to the given array of indices.

In [4]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

def encode(s):
    return [stoi[c] for c in s]

def decode(l):
    return ''.join([itos[i] for i in l])

In [5]:
encoded_hello = encode("hello")
decoded_hello = decode(encoded_hello)
print(encoded_hello)
print(decoded_hello)


[46, 43, 50, 50, 53]
hello


In our case, we are using a very simple technique/method. This has its tradeoffs. This simplicity will make it easier to get a "finished" product, and results in easier and less complex encode and decode functions. However, there are alternatives that use much larger vocabularies (using subphrases - I believe) that results in a smaller result when encoding. I am assuming that this results in an improvement to the speed of the model at the trade off of complexity and a larger vocabulary size (storage size).

In [6]:
%pip install torch


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch

/Volumes/2Tb-ext/Coding Projects/Python/AI/Gpt-model/venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


Now, we are encoding all the text in the little shakespeare dataset.


In [8]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


Let's split into a training and validation data set now.

In [9]:
split = int(0.85*len(data))

train_data = data[:split]
val_data = data[split:]


In [10]:
block_size = 8

train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range( block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} target: {target}')

when input is tensor([18]) target: 47
when input is tensor([18, 47]) target: 56
when input is tensor([18, 47, 56]) target: 57
when input is tensor([18, 47, 56, 57]) target: 58
when input is tensor([18, 47, 56, 57, 58]) target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target: 58
